In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot
from numpy import array
from numpy import hstack

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('seaborn-paper')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedShuffleSplit

import warnings
warnings.simplefilter('ignore', category=DeprecationWarning)

from keras.models import Model
from keras.layers import Permute
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

from utils.generic_utils import load_dataset_at, calculate_dataset_metrics, cutoff_choice, \
                                cutoff_sequence
from utils.constants import MAX_NB_VARIABLES, MAX_TIMESTEPS_LIST

from keras.models import Model
from keras.layers import Input, Dense, LSTM, multiply, concatenate, Activation, Masking, Reshape, GRU
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout

from utils.constants import MAX_NB_VARIABLES, NB_CLASSES_LIST, MAX_TIMESTEPS_LIST
from utils.constants import MAX_NB_VARIABLES, NB_CLASSES_LIST, MAX_TIMESTEPS_LIST
from utils.keras_utils import train_model, evaluate_model, set_trainable
from utils.layer_utils import AttentionLSTM

import time

DATASET_INDEX = 14

MAX_TIMESTEPS = MAX_TIMESTEPS_LIST[DATASET_INDEX]
MAX_NB_VARIABLES = MAX_NB_VARIABLES[DATASET_INDEX]
NB_CLASS = NB_CLASSES_LIST[DATASET_INDEX]

TRAINABLE = True

In [ ]:
#set directory path
path = "D:/Users/hjcam/Documents/Python Scripts/ECTE355 Project/"
os.chdir( path )

In [ ]:
data = {"Normal":{},"Aggressive":{},"Drowsy":{}}

#input directory with data file for importing

for behaviour in os.listdir('Dataset'):
    print(behaviour)
    for drive in os.listdir('Dataset/'+behaviour):
        dataset = []
        #load text data
        acc = np.loadtxt('Dataset/' + behaviour + '/' + drive + '/RAW_ACCELEROMETERS.txt')
        gps = np.loadtxt('Dataset/' + behaviour + '/' + drive + '/RAW_GPS.txt')
        lane = np.loadtxt('Dataset/' + behaviour + '/' + drive + '/PROC_LANE_DETECTION.txt')
        veh = np.loadtxt('Dataset/' + behaviour + '/' + drive + '/PROC_VEHICLE_DETECTION.txt')
        
        #select desired features
        acc_reduced = acc[:,[0,5,6,7,8,9,10]] #time, X KF,Y KF, Z KF, Roll, Pitch, Yaw
        gps_reduced = gps[:,[0,1]] #time, speed
        lane_reduced = lane[:,[0,1]] #time, lane center
        veh_reduced = veh[:,[0,1,2]] #time, dist. to veh, time to vehicle
        
        #transform in Pandas dataframe
        acc_data = pd.DataFrame(acc_reduced, columns = ['Time','X KF','Y KF', 'Z KF', 'Roll', 'Pitch', 'Yaw'])
        acc_data.set_index('Time',inplace = True)
        gps_data = pd.DataFrame(gps_reduced, columns = ['Time','Speed'])
        gps_data.set_index('Time',inplace = True)
        lane_data = pd.DataFrame(lane_reduced, columns = ['Time','Lane_X'])
        lane_data.set_index('Time',inplace = True)
        veh_data = pd.DataFrame(veh_reduced, columns = ['Time','Veh_Dist','Veh_Time'])
        veh_data.set_index('Time',inplace = True)
        
        acc_data = acc_data.groupby(acc_data.index).first()
        gps_data = gps_data.groupby(gps_data.index).first()
        lane_data = lane_data.groupby(lane_data.index).first()
        veh_data = veh_data.groupby(veh_data.index).first()
        
        #combine all datasets
        combined_data = pd.concat([acc_data, gps_data], axis=1, sort=False)
        
        #synchronise sampling rates
        combined_data['Speed'].interpolate(method = 'index',inplace = True)
        #combined_data['Lane_X'].interpolate(method = 'index',inplace = True)
        #combined_data['Veh_Dist'] = combined_data['Veh_Dist'].fillna(method='ffill')
        #combined_data['Veh_Time'] = combined_data['Veh_Time'].fillna(method='ffill')
        combined_data.dropna(inplace = True)
        combined_data = combined_data[['X KF','Y KF', 'Z KF', 'Roll', 'Pitch', 'Yaw','Speed']]#],'Lane_X','Veh_Dist','Veh_Time']]
        
        #output combined imported data to dictionary
        data[behaviour].update({drive:combined_data})

# Creating the overall formatted dataset

In [ ]:
#sliding window dataset split, input data, window size, behaviour and overlap
def split_sequences(sequences, n_steps, behaviour, n_overlap):
    X , y = list() , list()
    for i in np.arange(0,len(sequences),n_overlap,dtype = np.int16):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset stop
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x = sequences[i:end_ix, :]
        #X.append(list(map(list, zip(*seq_x))))
        X.append(seq_x)
        #insert class into seperate array
        if behaviour == 'Normal':
            y.append([0])
        elif behaviour == 'Drowsy':
            y.append([1])
        elif behaviour == 'Aggressive':
            y.append([2])
            
    return X , y

# Training the model

In [ ]:
#generate model, only requires shape of data set
def generate_model():
    ip = Input(shape=(x_train.shape[1], x_train.shape[2]))
    #LSTM component
    x = Masking()(ip)
    x = LSTM(16)(x)
    x = Dropout(0.2)(x)
    
    #FCN component
    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)
    #combine the two model architectures
    x = concatenate([x, y])

    out = Dense(3, activation='softmax')(x)

    model = Model(ip, out)
    #model.summary()

    # add load model code here to fine-tune

    return model

#squeeze excite block taken from literature

def squeeze_excite_block(input):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor
    Returns: a keras tensor
    '''
    filters = input._keras_shape[-1] # channel_axis = -1 for TF

    se = GlobalAveragePooling1D()(input)
    se = Reshape((1, filters))(se)
    se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    se = multiply([input, se])
    return se

In [ ]:
test_train_x = []
test_train_y = []

results_iter = {}

# set hyper parameters

steps = 300
overlap = 50

epochs=300
batch_size=128
counter = 0;

#create dataset through sliding window
        
for key in data:
    for drive in data[key]:
        x_windows, y_windows = split_sequences(data[key][drive].values, steps, key, overlap)
        test_train_x.extend(x_windows)
        test_train_y.extend(y_windows)
        
y = array(test_train_y)
x = array(test_train_x)

#normalise data
        
scaler = MinMaxScaler(feature_range=(0, 1))

for data_x in x:
    scaler.partial_fit(data_x)

scaled_x = []
        
# Second pass
for data_x in x:
    scaled_x.append(list(map(list, zip(*scaler.transform(data_x)))))
    
#create five folds for training
    
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=22)
sss.get_n_splits(array(scaled_x), array(y))

#train and test on each of the five folds
                 
for train_index, test_index in sss.split(array(scaled_x), array(y)):
    x_train, x_test = array(scaled_x)[train_index], array(scaled_x)[test_index]
    y_train_numbers, y_test_numbers = array(y)[train_index], array(y)[test_index]
    
    #set more hyper parameters and model checkpoints
    
    learning_rate= 1e-3
    monitor='loss'
    optimization_mode='auto'
    compile_model=True
    factor = 1. / np.cbrt(2)
    
    #location to save model weights
    
    weight_fn = "./weights/%s_weights.h5" % counter
    
    #reducing learning rate on a loss plateau
    
    reduce_lr = ReduceLROnPlateau(monitor=monitor, patience=100, mode=optimization_mode,
                                  factor=factor, cooldown=0, min_lr=1e-4, verbose=2)
    
    #save model weights on checkpoint if the loss is the minimal seen so far in all training epochs
    
    model_checkpoint = ModelCheckpoint(weight_fn, verbose=0, mode=optimization_mode,
                                       monitor=monitor, save_best_only=True, save_weights_only=True)
    callback_list = [reduce_lr, model_checkpoint]

    optm = Adam(lr=learning_rate)
    
    #generate model
        
    model = generate_model()
    model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
        
    classes = np.unique(y_train_numbers)
    le = LabelEncoder()
    y_ind = le.fit_transform(y_train_numbers.ravel())
    recip_freq = len(y_train_numbers) / (len(le.classes_) * np.bincount(y_ind).astype(np.float64))
    class_weight = recip_freq[le.transform(classes)]
        
    y_train = to_categorical(y_train_numbers, len(np.unique(y_train_numbers)))
    y_test = to_categorical(y_test_numbers, len(np.unique(y_test_numbers)))
    
    #train model

    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                class_weight=class_weight, verbose=0, callbacks=callback_list, validation_data=(x_test, y_test))
    
    #finished training, reload best weights
    
    model.load_weights(weight_fn)
    
    #time how long for classification
    
    start_time = time.time()
    
    #predict on test set
    
    y_pred = model.predict(x_test, batch_size=64, verbose=1)
    
    end_time = time.time()
    
    print('Total time: %s'%(end_time-start_time))
    
    #print the classification performance and save to array
    
    y_pred_bool = np.argmax(y_pred, axis=1)
    y_pred = to_categorical(y_pred_bool, len(np.unique(y_pred_bool)))
    print(classification_report(y_test, y_pred))
        
    results_iter[str(steps)+'_'+str(overlap)] = classification_report(y_test, y_pred)
    
    counter = counter + 1